In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import os
import tensorflow as tf
from keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
%matplotlib inline 

# Generate dummy data
import numpy as np

# https://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
# tensorboard
# https://www.youtube.com/watch?v=lV09_8432VA
# https://www.youtube.com/watch?v=BqgTU7_cBnk - stydy model

class TrainValTensorBoardF1(TensorBoard):
    def __init__(self, log_dir='./logs', name="test", **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, name)
        super(TrainValTensorBoardF1, self).__init__(training_log_dir, **kwargs)
        
        self.val_writer = tf.summary.FileWriter(training_log_dir)
        self.val_writer1 = tf.summary.FileWriter(training_log_dir)
        #self.val_writer_train = tf.summary.FileWriter(training_log_dir)

        
    def dump(self, f1_score_train, f1_score_cv, f1_score_test, x):
        
        # https://github.com/tensorflow/tensorflow/issues/7089
        
        # first
        from numpy import random
        
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = f1_score_train
        summary_value.tag = "f1_score_train"
        self.val_writer.add_summary(summary, x)

        # second
        if 1:
#             summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = f1_score_cv
            summary_value.tag = "f1_score_cv"
#             self.val_writer1.add_summary(summary, x)
            
        if 1:
#             summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = f1_score_test
            summary_value.tag = "f1_score_test"
            self.val_writer.add_summary(summary, x)
            
#         write_op = summary.merge_all()
      

#         log_var = tf.Variable(0.0)
#         tf.summary.scalar("loss", log_var)

        write_op = tf.summary.merge_all([f1_score_test, f1_score_train])

        #session = tf.InteractiveSession()
        #session.run(tf.global_variables_initializer())
        session = tf.keras.backend.get_session()

#         for i in range(100):
#             # for writer 1
#             summary = session.run(write_op, {log_var: random.rand()})
#             self.writer_1.add_summary(summary, i)
#             self.writer_1.flush()

#             # for writer 2
        summary = session.run(write_op, {log_var: random.rand()})
        self.val_writer.add_summary(summary, x)
#         self.writer_2.flush()
        
        self.val_writer.flush()
#         self.val_writer1.flush()
        
    def end_it(self):
        self.val_writer.close()
#         self.val_writer1.close()

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()
        
# FIXME: load our dataset
# load images
from roadmarks_pipelines import dataset_manager
from roadmarks_pipelines.dataset_manager import to_keras_y

path = '/train_samples/samples_ff128_neg_filtered'
dataset_root = '/mnt/d5/datasets_pa/'
    
data = dataset_manager.get_all_train_data(path, dataset_root)
#y, X, _ = dataset_manager.unpack_data(data)

# Load and split
size = len(data)
main_ratio = 0.8
print 'main_ratio:', main_ratio
t0 = int(size * main_ratio)
t1 = t0 + int(size * (1 - main_ratio) / 2)

y_train, x_train, fns_train = dataset_manager.unpack_data(data[0:t0])
y_cv, x_cv, fns_cv = dataset_manager.unpack_data(data[t0:t1])
y_test, x_test, _ = dataset_manager.unpack_data(data[t1:])

# conv
y_train = to_keras_y(y_train)
y_cv = to_keras_y(y_cv)
y_test = to_keras_y(y_test)

# FIXME: plain eval

# F1 as metric
# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras - seems bad idea for opt
input_dim = x_train.shape[1]
print "input_dim:", input_dim

from roadmarks_pipelines.Classifiers import FirstNFeaturesScaler
from roadmarks_pipelines.utils import F1_score_batched

# norm.
featrues_scaler = FirstNFeaturesScaler(input_dim)
featrues_scaler = featrues_scaler.fit(x_train)

# norm immid.
x_cv = featrues_scaler.transform(np.copy(x_cv))
x_test = featrues_scaler.transform(np.copy(x_test))
x_train = featrues_scaler.transform(np.copy(x_train))  

# params
decision_thr = 0.5
batch_size = 128

y_train_size = len(y_train)

key = 0

if key == 0:
    # Model
    tvtb = TrainValTensorBoardF1(name='m'+str(int(time.time())))
    for end_ptr in range(batch_size, y_train_size + batch_size * 2, batch_size * 2):
        ep = np.clip(end_ptr, 0, y_train_size-1)

        y_train_sample = np.copy(y_train[0:ep])
        if np.sum(y_train_sample) == 0:
            continue
        x_train_sample = np.copy(x_train[0:ep])
        
        print 'sz:', len(x_train_sample)

        # Build model
        model = Sequential()
        model.add(Dense(8, input_dim=input_dim, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        # Fit model
        history = model.fit(x_train_sample, y_train_sample,
                  epochs=2,
                  batch_size=batch_size, verbose=0)
        #, callbacks=[TrainValTensorBoard(write_graph=False)], 
         #                   )

        # Validation
        y_pred_test = np.array(model.predict(x_test) > decision_thr, dtype=np.int)


        # predict
        y_pred_train = np.array(model.predict(x_train_sample) > decision_thr, dtype=np.int)
        y_pred_cv = np.array(model.predict(x_cv) > decision_thr, dtype=np.int)

        f1_score_train = F1_score_batched(y_pred_train.ravel(), y_orig=y_train_sample.ravel())
        f1_score_cv = F1_score_batched(y_pred_cv.ravel(), y_orig=y_cv.ravel())
        f1_score_test = F1_score_batched(y_pred_test.ravel(), y_orig=y_test.ravel())  

        #f1_scores_train.append(f1_score_train)
        #f1_scores_cv.append(f1_score_cv)
        #x_axis.append(ep)
        
        tvtb.dump(f1_score_train, f1_score_cv, f1_score_test, ep)
        break

    tvtb.end_it()
    
elif key == 2:
    # https://www.kaggle.com/eikedehling/keras-nn-scaling-feature-selection-0-548
    pass
    


Loaded: /mnt/d5/datasets_pa//train_samples/samples_ff128_neg_filtered
pos: 162
neg: 2735
pos_aug: 1418
main_ratio: 0.8
input_dim: 128
sz: 128
F1:0.586 P:0.515 R:0.680
F1:0.578 P:0.504 R:0.676
F1:0.465 P:0.376 R:0.608
